In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10

In [2]:
data = pd.read_csv("./MC1/mc1-reports-data.csv",parse_dates=True,infer_datetime_format=True, index_col=0)
data = data.sort_index()
data = data.fillna(0)
data.to_csv("boxplot/data.csv")
data.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
time,,,,,,,
2020-04-06,2.0,4.0,1.0,0.0,4.0,0.0,17
2020-04-06,2.0,8.0,5.0,0.0,3.0,0.0,2
2020-04-06,9.0,7.0,5.0,0.0,4.0,0.0,8
2020-04-06,10.0,2.0,9.0,0.0,10.0,0.0,4
2020-04-06,6.0,2.0,9.0,0.0,4.0,1.0,15


#### Construcción dataset
- Se costruyen dos dataset, uno que contiene todos los datos para graficar los boxplot y otro para graficar los outliers.

In [3]:
l = 1
d = 8
data.loc[data["location"]==1].loc[data.loc[data["location"]==1].index.day==8]["shake_intensity"].max()

3.0

In [5]:
names = ["Palace Hills", "Northwest","Old Town", "Safe Town","Southwest","Downtown",
                       "Wilson Forest", "Scenic Vista", "Broadview", "Chapparal", "TerrapinSprings",
                       "Pepper Mill", "CheddardFord", "Easton", "Weston", "Southon", "Oak Willow",
                       "East Parton", "West Parton"]
localidade = []
tipo_daño = []
ldays = []
q1 = []
q2 = []
q3 = []
upper = []
lower = []
t_daño = ["sewer_and_water", "power", "roads_and_bridges", "medical", "buildings", "shake_intensity"]
for l in range(1,20):
    for d in range(6,12):
        for t in t_daño:
            localidade.append(l)
            tipo_daño.append(t)
            ldays.append(d)
            aux_q1 = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d][t].quantile(.25)
            aux_q2 = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d][t].quantile(.5)
            aux_q3 = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d][t].quantile(.75)
            q1.append(aux_q1)
            q2.append(aux_q2)
            q3.append(aux_q3)
            upper.append(aux_q3 + 1.5*(aux_q3-aux_q2))
            lower.append(aux_q1 - 1.5*(aux_q3-aux_q2))

In [97]:
len(localidade), len(tipo_daño), len(q1), len(q2), len(q3), len(upper), len(lower)

(684, 684, 684, 684, 684, 684, 684)

In [6]:
df_bxplt = pd.DataFrame()
df_bxplt["location"] = localidade
df_bxplt["day"] = ldays
df_bxplt["tipo"] = tipo_daño
df_bxplt["q1"] = q1
df_bxplt["q2"] = q2
df_bxplt["q3"] = q3
df_bxplt["upper"] = upper
df_bxplt["lower"] = lower
df_bxplt.iloc[:6]

,location,day,tipo,q1,q2,q3,upper,lower
0,1,6,sewer_and_water,0.0,4.0,8.0,14.0,-6.0
1,1,6,power,0.0,2.0,7.0,14.5,-7.5
2,1,6,roads_and_bridges,0.0,4.0,8.0,14.0,-6.0
3,1,6,medical,1.0,4.0,8.0,14.0,-5.0
4,1,6,buildings,0.0,4.0,8.0,14.0,-6.0
5,1,6,shake_intensity,0.0,0.0,0.0,0.0,0.0


In [7]:
#Para las localidades que quedaron sin datos
df_bxplt = df_bxplt.fillna(0)

In [8]:
df_bxplt.loc[df_bxplt.day==6].loc[df_bxplt.location==1]

,location,day,tipo,q1,q2,q3,upper,lower
0,1,6,sewer_and_water,0.0,4.0,8.0,14.0,-6.0
1,1,6,power,0.0,2.0,7.0,14.5,-7.5
2,1,6,roads_and_bridges,0.0,4.0,8.0,14.0,-6.0
3,1,6,medical,1.0,4.0,8.0,14.0,-5.0
4,1,6,buildings,0.0,4.0,8.0,14.0,-6.0
5,1,6,shake_intensity,0.0,0.0,0.0,0.0,0.0


- Como no se consideran valores negativos de magnitud, entonces los valores por debajo de cero pasan a ser iguales a cero.
- Tampoco se considerarán magnitud superiores a 10.

In [9]:
df_bxplt.loc[df_bxplt["lower"] < 0, "lower"]=0
df_bxplt.loc[df_bxplt["upper"] > 10, "upper"]=10
df_bxplt.to_csv("boxplot/data_boxplot.csv", index=False)

In [102]:
df_bxplt.head()

,location,day,tipo,q1,q2,q3,upper,lower
0,1,6,sewer_and_water,0.0,4.0,8.0,10.0,0.0
1,1,6,power,0.0,2.0,7.0,10.0,0.0
2,1,6,roads_and_bridges,0.0,4.0,8.0,10.0,0.0
3,1,6,medical,1.0,4.0,8.0,10.0,0.0
4,1,6,buildings,0.0,4.0,8.0,10.0,0.0


In [103]:
data.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
time,,,,,,,
2020-04-06,2.0,4.0,1.0,0.0,4.0,0.0,17
2020-04-06,2.0,8.0,5.0,0.0,3.0,0.0,2
2020-04-06,9.0,7.0,5.0,0.0,4.0,0.0,8
2020-04-06,10.0,2.0,9.0,0.0,10.0,0.0,4
2020-04-06,6.0,2.0,9.0,0.0,4.0,1.0,15


In [104]:
y_q1

array([3., 2., 3., 2., 2., 0.])

In [10]:
data = pd.read_csv("boxplot/data.csv",parse_dates=True,infer_datetime_format=True, index_col=0)
df_bxplt = pd.read_csv("boxplot/data_boxplot.csv")
x_ticks = ["sewer_and_water","power","roads_and_bridges","medical","buildings","shake_intensity"]
p = figure(plot_height=400, plot_width=800, title="Boxplot",background_fill_color="#efefef",
           tools="pan,reset,save,wheel_zoom, hover", x_range=x_ticks)
#reset_output()
output_notebook()
v_day = 6
v_loc = 3
#Para graficar las lineas superiores del boxplot
y_rupper = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["upper"]
p.rect(x_ticks, y_rupper, 0.2, 0.01, line_color="black")
#Para graficar las lineas inferiores del boxplot
y_rlower = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["lower"]
p.rect(x_ticks, y_rlower, 0.2, 0.01, line_color="black")
#Para graficar los segmentos del boxplot
y_q1 = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["q1"].values
y_q2 = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["q2"].values
y_q3 = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["q3"].values

source_seg = ColumnDataSource(dict(x=x_ticks, y_lower=y_rlower, y_upper=y_rupper, q1=y_q1, q2=y_q2, q3=y_q3))
p.segment("x", "y_lower", "x", "q1", line_color="black", source=source_seg)
p.segment("x", "y_upper", "x", "q3", line_color="black", source=source_seg)
#Para graficar las barras
p.vbar("x", 0.3, "q1", "q2", fill_color="#3B8686", line_color="black", source=source_seg)
p.vbar("x", 0.3, "q2", "q3", fill_color="#E08E79", line_color="black", source=source_seg)
#Para graficar los outliers si es que hay
aux_data = data[data.index.day==v_day].loc[data[data.index.day==v_day].location==v_loc]
for idx in range(aux_data.shape[0]):
    k = 0
    for col in x_ticks:
        if (aux_data.iloc[idx][col] < (y_q1[k] - 1.5*(y_q3[k]-y_q2[k])) or
            aux_data.iloc[idx][col] > (y_q3[k] + 1.5*(y_q3[k]-y_q2[k]))):
            p.circle([x_ticks[k]], aux_data.iloc[idx][col], size=6, color="#F38630", fill_alpha=0.6)
        k += 1
p.ygrid.grid_line_color = "white"
p.xgrid.grid_line_color = None
#output_file("/resultados/image25.html")
show(p)

Loading BokehJS ...

### Grafico interactivo
- Se corre con el servidor de bokeh.

In [11]:
day_value = 11
vec_value = 1
y_rupper = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["upper"].values
y_rlower = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["lower"].values
y_q1 = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["q1"].values
y_q2 = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["q2"].values
y_q3 = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["q3"].values
df_aux = pd.DataFrame()
df_aux["x"] = x_ticks
df_aux["y_lower"] = y_rlower
df_aux["y_upper"] = y_rupper
df_aux["q1"] = y_q1
df_aux["q2"] = y_q2
df_aux["q3"] = y_q3
df_aux

,x,y_lower,y_upper,q1,q2,q3
0,sewer_and_water,0.0,0.0,0.0,0.0,0.0
1,power,0.0,0.0,0.0,0.0,0.0
2,roads_and_bridges,0.0,0.0,0.0,0.0,0.0
3,medical,0.0,0.0,0.0,0.0,0.0
4,buildings,0.0,0.0,0.0,0.0,0.0
5,shake_intensity,0.0,0.0,0.0,0.0,0.0


In [210]:
a = df_out.groupby(["x_outs", "y_outs"], group_keys=False,as_index=False).last()

In [145]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange
from bokeh.models.annotations import Title
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout, gridplot
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10

#reset_output()
#output_notebook()
#nombres de los vecindarios
names = ["Palace Hills", "Northwest","Old Town", "Safe Town","Southwest","Downtown",
         "Wilson Forest", "Scenic Vista", "Broadview", "Chapparal", "TerrapinSprings",
         "Pepper Mill", "CheddardFord", "Easton", "Weston", "Southon", "Oak Willow",
         "East Parton", "West Parton"]
#se cargan los datasets
data = pd.read_csv("boxplot/data.csv",parse_dates=True,infer_datetime_format=True, index_col=0)
df_bxplt = pd.read_csv("boxplot/data_boxplot.csv")
df_outs = pd.read_csv("boxplot/df_outs.csv")

x_ticks = ["sewer_and_water","power","roads_and_bridges","medical","buildings","shake_intensity"]
p = figure(plot_height=400, plot_width=800, title="Boxplot",background_fill_color="#efefef",
           tools="pan,reset,save,wheel_zoom, hover", x_range=x_ticks, y_range=(-1,11))

p2 = figure(plot_height=300, plot_width=300, title="quantidade de outliers",
           tools="pan,reset,save,wheel_zoom, hover", x_range=x_ticks)
p2.xaxis.major_label_orientation = "vertical"

v_day = 6 #Configuración inicial del dia
v_loc = 1 #Configuración inicial del vecindario
y_rupper = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["upper"].values
y_rlower = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["lower"].values
y_q1 = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["q1"].values
y_q2 = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["q2"].values
y_q3 = df_bxplt.loc[df_bxplt.day==v_day].loc[df_bxplt.location==v_loc]["q3"].values
source_seg = ColumnDataSource(dict(x=x_ticks, y_lower=y_rlower, y_upper=y_rupper, q1=y_q1, q2=y_q2, q3=y_q3))
#Para graficar las lineas superiores del boxplot
p.rect("x", "y_upper", 0.2, 0.01, line_color="black", source=source_seg)
#Para graficar las lineas inferiores del boxplot
p.rect("x", "y_lower", 0.2, 0.01, line_color="black", source=source_seg)
#Para graficar los segmentos del boxplot
p.segment("x", "y_lower", "x", "q1", line_color="black", source=source_seg)
p.segment("x", "y_upper", "x", "q3", line_color="black", source=source_seg)
#Para graficar las barras
p.vbar("x", 0.3, "q1", "q2", fill_color="#3B8686", line_color="black", source=source_seg)
p.vbar("x", 0.3, "q2", "q3", fill_color="#E08E79", line_color="black", source=source_seg)
#Para graficar los outliers si es que hay
#para pintar solo de a un punto si se presentan los mismos datos
aux_data = df_outs.groupby(["day","vecindario","x_outs","y_outs"],as_index=False).last().reset_index(drop=True)

source_out = ColumnDataSource(dict(aux_data[aux_data.day==v_day].loc[aux_data[aux_data.day==v_day].vecindario==v_loc]))
p.circle("x_outs", "y_outs", size=6, color="#F38630", fill_alpha=0.6, source=source_out)

p.ygrid.grid_line_color = "white"
p.xgrid.grid_line_color = None

#PARA EL GRÁFICO DE BARRAS QUE LLEVA LA CANTIDAD DE OUTLIERS POR CADA TIPO
num_outs = []
df_outs_aux = df_outs.loc[df_outs.vecindario==1].loc[df_outs.loc[df_outs.vecindario==1].day==6]
for t in x_ticks:
    num_outs.append(df_outs_aux.loc[df_outs_aux.x_outs==t].shape[0])
source_nout = ColumnDataSource(dict(x_outs=x_ticks, y_outs=num_outs))
p2.vbar("x_outs", 0.3, "y_outs", fill_color="#3B8686", line_color="black", source=source_nout)
p2.hover.tooltips = [("quantidade", "@y_outs")]

#Barras de interacción
s_day_text = "registro de acordo com o dia"
s_day = Slider(title=s_day_text, value=6.0, start=6.0, end=11.0, step=1.0)
s_vec_text = "vecindario"
s_vec= Slider(title=s_vec_text, value=1.0, start=1.0, end=19.0, step=1.0)

def update_data(attrname, old, new):
    day_value = s_day.value
    vec_value = s_vec.value
    y_rupper = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["upper"].values
    y_rlower = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["lower"].values
    y_q1 = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["q1"].values
    y_q2 = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["q2"].values
    y_q3 = df_bxplt.loc[df_bxplt.day==day_value].loc[df_bxplt.location==vec_value]["q3"].values
    df_aux = pd.DataFrame()
    df_aux["x"] = x_ticks
    df_aux["y_lower"] = y_rlower
    df_aux["y_upper"] = y_rupper
    df_aux["q1"] = y_q1
    df_aux["q2"] = y_q2
    df_aux["q3"] = y_q3
    source_seg.data = dict(df_aux)
    #Para actualizar los datos de los outliers
    source_out.data = dict(aux_data[aux_data.day==day_value].loc[aux_data[aux_data.day==day_value].vecindario==vec_value])
    #Para actualizar el gráfico de barras
    num_outs = []
    df_outs_aux = df_outs.loc[df_outs.vecindario==vec_value].loc[df_outs.loc[df_outs.vecindario==vec_value].day==day_value]
    for t in x_ticks:
        num_outs.append(df_outs_aux.loc[df_outs_aux.x_outs==t].shape[0])
    source_nout.data = dict(x_outs=x_ticks, y_outs=num_outs)
    
    
for w in [s_day, s_vec]:
    w.on_change('value', update_data)

# Set up layouts and add to document
inputs = column([s_day,s_vec], width=200)

curdoc().add_root(row(inputs, gridplot([[p, p2]]), width=400))

show(p)

Loading BokehJS ...

In [35]:
df_outs = pd.DataFrame(columns=["vecindario","dia","x_outs","y_outs"])

In [44]:
df_outs = df_outs.append(pd.DataFrame([[1,3,4,5]],columns=["vecindario","dia","x_outs","y_outs"]))

In [45]:
df_outs

,vecindario,dia,x_outs,y_outs
0,1,3,4,5
0,1,3,4,5
0,1,3,4,5


#### Creando DataSets de Outliers
-Para no estar calculandolos, se genera desde un comienzo el dataset que contiene los outliers de cada vecindario.

In [5]:
data["day"] = data.index.day
data.head()

In [53]:
a = np.array([1,2,3,4]).reshape(4,1)
b = np.array([5,6,7,8]).reshape(4,1)
np.concatenate((a,b), axis=1)

(4, 2)

In [84]:
np.array([[1,2,3],[4,5,6]])

array([[1, 2, 3],
       [4, 5, 6]])

In [96]:
data.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location,day
time,,,,,,,,
2020-04-06,2.0,4.0,1.0,0.0,4.0,0.0,17,6
2020-04-06,2.0,8.0,5.0,0.0,3.0,0.0,2,6
2020-04-06,9.0,7.0,5.0,0.0,4.0,0.0,8,6
2020-04-06,10.0,2.0,9.0,0.0,10.0,0.0,4,6
2020-04-06,6.0,2.0,9.0,0.0,4.0,1.0,15,6


In [99]:
x_ticks = ["sewer_and_water","power","roads_and_bridges","medical","buildings","shake_intensity"]
df_outs = pd.DataFrame(columns=["vecindario","day","x_outs","y_outs"])
for l in range(1, 20):
    for d in range(6,12):
        for t in x_ticks:
            data_aux = data[data.index.day==d].loc[data[data.index.day==d].location==l]
            #cálculo de los quartiles de cada tipo de daño
            q_1 = data_aux[t].quantile(.25)
            q_2 = data_aux[t].quantile(.50)
            q_3 = data_aux[t].quantile(.75)
            #para sacar los indices de los outliers
            l_s = q_3 + 1.5*(q_3 - q_1)
            l_i = q_1 - 1.5*(q_3 - q_1)
            out_upper = data_aux.loc[data_aux[t].values > l_s]
            out_lower = data_aux.loc[data_aux[t].values < l_i]
            #print(out_upper)
            add_upper = pd.DataFrame()
            add_upper["vecindario"] = out_upper.location.values
            add_upper["day"] = out_upper.day.values
            add_upper["x_outs"] = t
            add_upper["y_outs"] = out_upper[t].values
            df_outs = df_outs.append(add_upper)
            add_lower = pd.DataFrame()
            add_lower["vecindario"] = out_lower.location.values
            add_lower["day"] = out_lower.day.values
            add_lower["x_outs"] = t
            add_lower["y_outs"] = out_lower[t].values
            df_outs = df_outs.append(add_upper)

In [106]:
df_outs = df_outs.reset_index(drop=True)
df_outs.to_csv("./boxplot/df_outs.csv", index=False)

In [118]:
#para pintar solo de a un punto si se presentan los mismos datos
df_outs.groupby(["day","vecindario","x_outs","y_outs"],as_index=False).last().reset_index(drop=True)

,vecindario,day,x_outs,y_outs
0,1,6,shake_intensity,1.0
1,2,6,medical,2.0
2,2,6,medical,3.0
3,2,6,medical,4.0
4,2,6,medical,7.0
5,2,6,medical,10.0
6,2,6,shake_intensity,1.0
7,3,6,buildings,8.0
8,3,6,buildings,9.0
9,3,6,buildings,10.0


In [122]:
aux_data = df_outs.groupby(["day","vecindario","x_outs","y_outs"],as_index=False).last().reset_index(drop=True)
aux_data[aux_data.day==6].loc[aux_data[aux_data.day==6].vecindario==2]

,vecindario,day,x_outs,y_outs
1,2,6,medical,2.0
2,2,6,medical,3.0
3,2,6,medical,4.0
4,2,6,medical,7.0
5,2,6,medical,10.0
6,2,6,shake_intensity,1.0


In [144]:
df_outs_aux = df_outs.loc[df_outs.vecindario==1].loc[df_outs.loc[df_outs.vecindario==1].day==6]
df_outs_aux.loc[df_outs_aux.x_outs=="medical"]

,vecindario,day,x_outs,y_outs
